In [4]:
from transformers import DonutProcessor
import sys
import json
import os
sys.path.insert(1, 'PubTabNet/src/')
sys.path.insert(1, '../proj/src/')

from metric import TEDS
from processing_tabeleiro import TabeleiroProcessor

SPLIT = "val/"

ANN_PATH = "data/anns/"

json_list = os.listdir(ANN_PATH + SPLIT)

/home/jao/Documents/UFLA/Mestrado/venv_mestrado/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
aux_list = []
html_list = []

for f in json_list:
    if(f[-6:-5] == "L"):
        html_list.append(f)
    else:
        aux_list.append(f)
json_list = aux_list

In [23]:
tml_anns_list = []
for file in json_list:
    with open(ANN_PATH + SPLIT+file) as f:
        tml_anns_list.append(json.load(f))

In [24]:
from transformers import DonutProcessor

tml_processor = TabeleiroProcessor.from_pretrained("processors/Donut_PubTables_TML_Processor8k", crop_right=False)

In [25]:
tml_code_list = [processor.json2token(ann) for ann in anns_list]
tml_code_list[0]

'<table><row><col_header><b>Factor</b><col_header><b>Total</b><col_header><b>ARI</b><col_header><b>Odds ratios</b><col_header><b>95% CI</b><col_header><b><i>P</i>-value</b><row><span_type=1000><span_type=1001><span_type=1001><span_type=1001><span_type=1001><span_type=0001><content>Exposure to wood smoke<row><cell> Not exposed<cell>203<cell>82(40.39%)<cell>1<cell><cell><row><cell> Exposed<cell>309<cell>198(64.08%)<cell>2.63<cell>1.83–3.79<cell>< 0.001<row><span_type=1000><span_type=1001><span_type=1001><span_type=1001><span_type=1001><span_type=0001><content>Cigarette smoke<row><cell> Non passive smoker<cell>474<cell>249(52.53%)<cell>1<cell><cell><row><cell> Passive smoker<cell>37<cell>31(83.78%)<cell>4.67<cell>1.91–11.40<cell>< 0.001<row><span_type=1000><span_type=1001><span_type=1001><span_type=1001><span_type=1001><span_type=0001><content>History of contact<sup>a</sup><row><cell> No contact<cell>313<cell>134(42.81%)<cell>1<cell><cell><row><cell> Had contact<cell>199<cell>147(73.87%)<

In [29]:
tml_token_lens = []
n = 10
for code_chunk in [tml_code_list[i:i + len(tml_code_list)//n] for i in range(0, len(tml_code_list), len(tml_code_list)//n)]:
    input_ids_list = tml_processor.tokenizer(
            code_chunk,
            add_special_tokens=False,
            max_length= 10000,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )["input_ids"]
    for input_ids in input_ids_list:
        tml_token_lens.append(int((input_ids == 1).nonzero(as_tuple=True)[0][0]))

In [38]:
import numpy as np

print(np.max(tml_token_lens))
print(np.mean(tml_token_lens))
print(np.percentile(tml_token_lens, 99.4))

2782
346.3012286090391
1473.100000000013


In [31]:
html_anns_list = []
for file in html_list:
    with open(ANN_PATH + SPLIT+file) as f:
        html_anns_list.append(json.load(f))

In [32]:
html_processor = DonutProcessor.from_pretrained("processors/Donut_PubTables_HTML_Processor8k")

In [39]:
html_token_lens = []
n = 10
for code_chunk in [html_anns_list[i:i + len(html_anns_list)//n] for i in range(0, len(html_anns_list), len(html_anns_list)//n)]:
    input_ids_list = html_processor.tokenizer(
            code_chunk,
            add_special_tokens=False,
            max_length= 10000,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )["input_ids"]
    for input_ids in input_ids_list:
        html_token_lens.append(int((input_ids == 1).nonzero(as_tuple=True)[0][0]))

In [47]:
print(np.max(html_token_lens))
print(np.mean(html_token_lens))
print(np.percentile(html_token_lens, 97.4))

6083
517.3418540866703
1533.036
